In [4]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbick(RootModel):
    root:list[Site]


data = Youbick.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-18 22:41:18,復興南路二段235號前,True,28,4,25.02605,121.54360,24
1,復興南路二段273號前,大安區,2024-06-18 22:35:20,復興南路二段273號西側,True,21,10,25.02565,121.54357,11
2,國北教大實小東側門,大安區,2024-06-18 21:59:14,和平東路二段96巷7號,True,16,16,25.02429,121.54124,0
3,和平公園東側,大安區,2024-06-18 22:38:15,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
4,辛亥復興路口西北側,大安區,2024-06-18 22:42:19,復興南路二段368號,True,16,13,25.02153,121.54299,3
...,...,...,...,...,...,...,...,...,...,...
1422,臺大總圖書館西南側,臺大公館校區,2024-06-18 21:04:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1423,臺大黑森林西側,臺大公館校區,2024-06-18 22:35:20,臺大霖澤館南側,True,20,4,25.01995,121.54347,16
1424,臺大獸醫館南側,臺大公館校區,2024-06-18 22:33:17,臺大獸醫系館南側,True,24,0,25.01791,121.54242,24
1425,臺大新體育館東南側,臺大公館校區,2024-06-18 22:28:14,臺大體育館東側,True,40,2,25.02112,121.53591,38


In [2]:
df[df['可借']<=3]

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
9,新生南路三段82號前,大安區,2024-06-18 22:19:20,新生南路三段82號,True,20,3,25.018940,121.533610,17
10,辛亥路一段30號前,大安區,2024-06-18 22:16:20,辛亥路一段30號,True,8,3,25.019860,121.529820,5
13,大安運動中心停車場,大安區,2024-06-18 22:17:28,敦南街76巷28號,True,14,1,25.020348,121.546446,13
14,羅斯福路三段245號前,大安區,2024-06-18 21:58:20,羅斯福路三段245號,True,8,3,25.019270,121.529890,5
15,溫州公園,大安區,2024-06-18 22:16:20,羅斯福路三段283巷11號,True,5,3,25.018950,121.531560,2
...,...,...,...,...,...,...,...,...,...,...
1421,臺大圖資系館北側,臺大公館校區,2024-06-18 21:23:14,臺大圖書資訊館北側,True,25,2,25.018040,121.539280,23
1422,臺大總圖書館西南側,臺大公館校區,2024-06-18 21:04:14,臺大圖書館西南側,True,30,0,25.016900,121.540310,30
1423,臺大黑森林西側,臺大公館校區,2024-06-18 22:19:15,臺大霖澤館南側,True,20,0,25.019950,121.543470,20
1424,臺大獸醫館南側,臺大公館校區,2024-06-18 21:52:14,臺大獸醫系館南側,True,24,0,25.017910,121.542420,24


In [3]:
df.info()
#因為有使用pydantic所以基本上不用再進行檢查，但後須還是要使用

NameError: name 'df' is not defined

In [ ]:
#query是使用布林表達式查詢 DataFrame 的列，直接寫欄位名及要查詢的條件。
#query 
df.query('可借<=3')

In [6]:
import numpy as np

less3_df = df.query('可借<=3')
less3_df.shape

(465, 10)

In [ ]:
df2=df.query('可借<=3 and 可還<=30')
df2

In [10]:
#記得要搜尋的行政區域名稱要用""進行區分
df3=df.query('行政區域=="大安區" and 可借==0')
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
13,大安運動中心停車場,大安區,2024-06-18 22:23:07,敦南街76巷28號,True,14,0,25.020348,121.546446,14
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.533100,5
31,臺大動物醫院前,大安區,2024-06-18 22:42:19,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.542910,18
57,捷運古亭站(6號出口),大安區,2024-06-18 22:42:15,羅斯福路二段41號前,True,60,0,25.027600,121.522170,60
67,敦化南路二段293巷口南側,大安區,2024-06-18 22:34:15,敦化南路二段293巷口南側,True,14,0,25.023700,121.548990,14
76,復興南路1段340巷口,大安區,2024-06-18 22:29:14,復興南路一段信義路三段147巷45弄口,True,50,0,25.037010,121.543480,50
77,建國濟南路口,大安區,2024-06-18 22:21:14,建國南路一段/濟南路三段口(東南側),True,54,0,25.039350,121.538280,54
78,捷運忠孝復興站(2號出口),大安區,2024-06-18 22:37:15,復興南路一段/仁愛路三段123巷13弄(西北側),True,43,0,25.040350,121.543600,43
82,龍門廣場,大安區,2024-06-18 22:31:15,敦化南路一段與敦化南路一段236巷口,True,70,0,25.040920,121.548250,70
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.042010,121.546120,20


In [12]:
n=1
df4=df.query('行政區域=="大安區" and 可借==@n')
df4

#可以透過在環境變數前面加上“@”字元（例如@a + b）來引用它們。

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
26,臺大環研大樓,大安區,2024-06-18 22:31:20,芳蘭路/基隆路三段155巷口,True,15,1,25.01234,121.54503,14
58,捷運古亭站(5號出口),大安區,2024-06-18 22:43:19,和平東路一段19號前,True,14,1,25.02715,121.52293,13
72,台北科技大學億光大樓,大安區,2024-06-18 22:37:20,忠孝東路三段81號,True,45,1,25.04177,121.53833,44
81,信義大安路口(信維大樓),大安區,2024-06-18 22:25:21,信義路四段74號(前),True,18,1,25.03317,121.54639,17
94,捷運忠孝新生站(6號出口),大安區,2024-06-18 22:40:29,新生南路一段101號,True,15,1,25.04091,121.53295,14
102,東豐公園,大安區,2024-06-18 22:42:19,東豐街24號(旁公園路側),True,19,1,25.03569,121.54510,18
105,捷運忠孝敦化站(2號出口),大安區,2024-06-18 22:43:19,忠孝東路四段221號,True,17,1,25.04153,121.55240,16
125,臺北科技大學(電機工程系),大安區,2024-06-18 22:36:20,忠孝東路三段100號對側,True,16,1,25.04196,121.53653,15
129,捷運大安站(6號出口),大安區,2024-06-18 22:33:22,復興南路一段303號前,True,19,1,25.03479,121.54376,18
132,臥龍基隆路口,大安區,2024-06-18 22:40:23,臥龍街與基隆路二段口東北側,True,16,1,25.02039,121.54853,15
